# Forward Modeling MOXSI Data

Some brief notes on forward modeling MOXSI data such that each step is explicitly spelled out. At a high level, the steps for computing a MOXSI image are as follows:

1. Compute a DEM
2. Fold DEM through spectra from CHIANTI
3. Multiply the resulting spectra by the instrument wavelength response function
4. Reproject the spectral cube to the detector plane

Below, more details are provided about each step. After each step, we will denote the resulting quantity and the associated units as denoted in $[]$.

## 1. Compute differential emission measure (DEM)

Using a combination of AIA and XRT data, compute a differential emission measure (DEM) distribution for the full sun.
Before running the DEM inversion code, each image is reprojected to the same WCS such that all images are aligned.
Note that the images need not be at the exact detector resolution, but they should be below the resolution and should include as large a FOV as possible (full sun at minimum).
Additionally, the AIA images are corrected for any instrument degradation and all images are normalized to the exposure time.

As such, the differential emission measure will be a function of both space and temperature and will be denoted as

$$
\xi(x,y,T)\quad\quad[\mathrm{cm}^{-5}\mathrm{K}^{-1}]
$$

## 2. Compute intensity

### 2a. Get spectral table from CHIANTI

To transform $\xi$ into intensity, we first compute a set of isothermal spectra using the CHIANTI database.
This is done using the `ch_synthetic` and `make_chianti_spec` routines in CHIANTI IDL.
The temperature range is chosen to include all temperatures in the calculation from step 1.
It is assumed that the pressure is constant at $10^{15}$ cm$^{-3}$ K such that density varies inversely with $T$.
The spectral range is chosen such that it fully includes the bandpass of the instrument.
We include all continuum contributions (free-free, free-bound, two-photon) in the spectrum as well as all lines from the database.

The resulting spectral table is denoted as

$$
\mathcal{I}(\lambda, T)\quad\quad[\mathrm{cm}^3\mathrm{ph}\,\mathrm{Å}^{-1}\mathrm{s}^{-1}\mathrm{sr}^{-1}]
$$

### 2b. Compute intensity from spectra and DEM

To convert our DEM into intensity, we integrate the spectral table over $T$ weighted by $\xi$.
This gives us intensity, in physical units, at each spatial location as a function of wavelength, 

$$
I(x, y, \lambda) = \int_T\mathrm{d}T\,\xi(x,y,T)\mathcal{I}(\lambda, T)\quad\quad[\mathrm{ph}\,\mathrm{cm}^{-2}\mathrm{Å}^{-1}\mathrm{s}^{-1}\mathrm{sr}^{-1}]
$$

## 3. Convert to detector units

### 3a. Effective Area

To convert from physical units to detector units, we use the effective area of the instrument.
The effective area is dependent on wavelength $\lambda$ as well as the particular spectral order $\mu$ we are looking at,

$$
A_{eff}(\mu,\lambda) = A_{geo}T_F(\lambda)E_D(\lambda)E_G(\lambda,\mu) \quad\quad [\mathrm{cm}^2]
$$

where $A_{geo}$ is the geometrical collecting area of the pinhole, $T_F$ is the filter transmission, $E_D$ is the detector efficiency, and $E_G$ is the efficiency of the grating.
Note that $E_G$ varies with spectral order $\mu$.

### 3b. Gain

Additionally, to convert from photons to DN, we can multiply by the gain,

$$
g(\lambda) = \frac{g_{camera}}{3.65}\frac{hc}{\lambda}\quad\quad[\mathrm{DN}\,\mathrm{ph}^{-1}]
$$

### 3c. Plate Scale

We need to also multiply by the spatial and spectral plate scales,

$$
\delta_{xy}\quad\quad[\mathrm{sr}\,\mathrm{pix}^{-1}]
$$
$$
\delta_\lambda\quad\quad[\mathrm{Å}]
$$

### 3d. Combined wavelength response

Thus, to convert to physical units, we combine all of these factors together to convert to a flux in detector units,

$$
p(x,y,\lambda,\mu) = I(x,y,\lambda)A_{eff}(\lambda,\mu)g(\lambda)\delta_{xy}\delta_{\lambda} \quad\quad [\mathrm{DN}\,\mathrm{s}^{-1}\mathrm{pix}^{-1}]
$$

Note that if we want our flux in photons rather than DN, we drop the gain term $g$,

$$
p^{\prime}(x,y,\lambda,\mu) = I(x,y,\lambda)A_{eff}(\lambda,\mu)\delta_{xy}\delta_{\lambda} \quad\quad [\mathrm{ph}\,\mathrm{s}^{-1}\mathrm{pix}^{-1}]
$$

Also note that this is specific to a given spectral order $\mu$ because of the dependence in the effective area term

## 4. Reproject to detector geometry

We can now transform our spectral cube coordinates $(x,y,\lambda,\mu)$ into detector coordinates $(p_1, p_2, p_3)$.
Note that the final coordinate $p_3$ is actually degenerate as we will sum along the $p_3$ axis to create our overlapped image.
However, our coordinate system will still vary over $p_3$.
That is, our particular world coordinate system is dependent on a chosen $\lambda$ as denoted by $p_3$.

We transform our grid according to the following `PC_ij` matrix,

$$
M = R(-(\alpha-\gamma))D(\mu)R(-\gamma)
 = \begin{bmatrix}
\cos{\left(\alpha \right)} & \sin{\left(\alpha \right)} & - \mu \sin{\left(\alpha - \gamma \right)}\\
-\sin{\left(\alpha \right)} & \cos{\left(\alpha \right)} & - \mu \cos{\left(\alpha - \gamma \right)}\\
0 & 0 & 1
\end{bmatrix}
$$

where $\alpha$ denotes the roll of the instrument and $\gamma$ the direction of the dispersion across the detector.

To map our coordinates from our spectral cube to the detector geometry, we construct a WCS using the above `PC_ij` and reproject from the WCS of our spectral cube to the WCS of the detector.
We perform this operation for each spectral order ($\mu=-3, -1, 0, +1, +3$) and then sum all orders.
Note that Step 3 (above) must be performed for each spectral order as well.